### Libray Import

필요한 라이브러리를 import 합니다.

In [1]:
import cv2
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import numpy as np
import torch
import torch.fft as fft
import random
import timm
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, transforms
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR

def set_seed(seed=None):
    """모든 랜덤 시드를 고정하는 함수."""
    random.seed(seed)  # Python 내장 랜덤 모듈
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # CPU를 위한 PyTorch 시드
    torch.cuda.manual_seed(seed)  # GPU를 위한 PyTorch 시드
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 위한 PyTorch 시드
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python 해시 기반 작업을 위한 시드

    # PyTorch의 일관된 작업을 위해 다음 설정을 고려할 수 있습니다
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [2]:
set_seed(42)
if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device("cuda")
else:
    print("GPU is not available. Using CPU instead.")
    device = torch.device("cpu")

GPU is available.


In [3]:
def extract_frames(video_path, frame_count=1, resize=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    
    step = max(1, np.floor(total_frames / frame_count).astype(int))

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR by default
        frames.append(frame)

    cap.release()
    return frames

In [4]:
class VideoDataset(Dataset):
    def __init__(self, directory, frame_count=1, resize=(224, 224), transform=None, mode='train'):
        self.directory = directory
        self.frame_count = frame_count
        self.resize = resize
        self.transform = transform
        self.mode = mode
        self.classes = ['real', 'fake']
        self.videos = []

        if mode in ['train', 'val']:
            for label in self.classes:
                class_dir = os.path.join(directory, label)
                for video in os.listdir(class_dir):
                    self.videos.append((os.path.join(class_dir, video), label))
        elif mode == 'test':
            for video in directory:
                #                 print(video)
                self.videos.append((video, None))

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_path, label = self.videos[idx]
        video_name = os.path.basename(video_path)  # 비디오 파일 이름 추출
        frames = extract_frames(video_path, self.frame_count, self.resize)

        if self.transform:
            frames = [self.transform(frame) for frame in frames]
            fft_frames = [torch.abs(fft.fft2(frame, dim=(1, 2), norm='ortho')) for frame in frames]

        frames = np.stack(frames)
        frames = torch.from_numpy(frames).float()
        frames = frames.squeeze()

        fft_frames = np.stack(fft_frames)
        fft_frames = torch.from_numpy(fft_frames).float()
        fft_frames = fft_frames.squeeze()

        if self.mode in ['train', 'val']:
            label = torch.tensor(self.classes.index(label))
        elif self.mode == 'test':
            return frames, fft_frames  

        return frames, fft_frames, label

### 모델링

학습데이터를 이용하여 자유롭게 모델링 해보세요!

In [5]:
## model for fft domain.
class VideoClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super(VideoClassifier, self).__init__()
        self.img_model = timm.create_model('xception', pretrained=True)
        self.fft_model = timm.create_model('swinv2_small_window8_256.ms_in1k', pretrained=True)
        in_ch_img = self.img_model.fc.in_features
        in_ch_fft = self.fft_model.head.in_features
        self.img_model.fc = nn.Identity()
        self.fft_model.head.fc = nn.Identity()
        
        self.out = nn.Linear(in_ch_img + in_ch_fft, 2)

    def forward(self, img, fft):
        img_f = self.img_model(img)
        fft_f = self.fft_model(fft)
        x = torch.cat((fft_f, img_f), dim=1)
        return self.out(x)

In [6]:
TRAIN_PATH = '/mnt/elice/dataset/train'
TEST_PATH = '/mnt/elice/dataset/test'

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(256),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 데이터셋 및 데이터 로더 설정
dataset = VideoDataset(directory=TRAIN_PATH, transform=transform)
total_size = len(dataset)
train_size = int(total_size * 0.7)
val_size = total_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


print(total_size,train_size,val_size )

## model #1

In [ ]:
model_resnet50 = timm.create_model('res2net50_26w_4s', pretrained=True, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet50.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_resnet50.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, _, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_resnet50.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, _, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnet50(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_resnet50.state_dict(), 'resnet_best_model_0.9726.pt')

## model #2

In [ ]:
model_inc4 = timm.models.inception_v4(pretrained=True,num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_inc4.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_inc4.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, _, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_inc4(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_inc4.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, _, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_inc4(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_inc4.state_dict(), 'incv4_model_0.974.pt')

## model #3

In [ ]:
model_xception = timm.create_model('xception', pretrained=True, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_xception.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_xception.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, _, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_xception(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_xception.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, _, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_xception(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_xception.state_dict(), 'xception_best_model_0.974.pt')

## model #4

In [ ]:
model_resnext50d_32x4d = timm.models.resnext50d_32x4d(pretrained=True,num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnext50d_32x4d.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_resnext50d_32x4d.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, _, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnext50d_32x4d(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_resnext50d_32x4d.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, _, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnext50d_32x4d(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_resnext50d_32x4d.state_dict(), 'resnext_best_model_0.975.pt')

## model #5

In [ ]:
model_renest50d = timm.create_model('resnest50d.in1k', pretrained=True, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_renest50d.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_renest50d.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, _, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_renest50d(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_renest50d.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs, _, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_renest50d(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_renest50d.state_dict(), 'resnest_best_model_0.976.pt')

## model #6

In [ ]:
model_twin = VideoClassifier(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_twin.parameters(), lr=0.0001)

num_epochs = 10
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
best_val_loss = float('inf')

In [ ]:
for epoch in range(num_epochs):
    model_twin.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        img_inputs, fft_img, labels = data
        img_inputs, fft_img,labels = img_inputs.to(device),fft_img.to(device),  labels.to(device)

        optimizer.zero_grad()
        outputs = model_twin(img_inputs, fft_img)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= train_total

    model_twin.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            img_inputs, fft_img, labels = data
            img_inputs, fft_img,labels = img_inputs.to(device),fft_img.to(device),  labels.to(device)
            outputs = model_twin(img_inputs, fft_img)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    # 에포크가 끝날 때마다 스케줄러 업데이트
    scheduler.step()

    # 현재 학습률 출력
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current Learning Rate: {current_lr:.6f}")

    # 검증 손실이 이전의 최고값보다 낮으면 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_twin.state_dict(), 'best_model_twin_0.972.pt')

## 추론 시작

In [8]:
from glob import glob
test_files = sorted(glob(TEST_PATH+'/*'))
test_files

['/mnt/elice/dataset/test/000ee59f529d4d069edee33b887bb2ce.mp4',
 '/mnt/elice/dataset/test/0018214f560f4ba3871dd9218f55dd48.mp4',
 '/mnt/elice/dataset/test/00241f732f934e2b807cf493d02ff80b.mp4',
 '/mnt/elice/dataset/test/00473872f36940a0bfe3f3dfd7667449.mp4',
 '/mnt/elice/dataset/test/004d9c5bee68438397889c71a32a05d5.mp4',
 '/mnt/elice/dataset/test/005dfa81cca84c03a0ecbf6d615af3a1.mp4',
 '/mnt/elice/dataset/test/005ed57ae7a34ba392f55bcbb3343440.mp4',
 '/mnt/elice/dataset/test/008a9931aed04545adda460e972547e5.mp4',
 '/mnt/elice/dataset/test/009fab774fb241d8910323674dedef1c.mp4',
 '/mnt/elice/dataset/test/00cc2ff2ff78411aa9f4e472589f1f22.mp4',
 '/mnt/elice/dataset/test/00d061fd346749a985a11eebdd16489c.mp4',
 '/mnt/elice/dataset/test/00f0a2639d3547fca2b52f45ea153189.mp4',
 '/mnt/elice/dataset/test/0102a1124d124527a9022c2d8288332f.mp4',
 '/mnt/elice/dataset/test/0106e97f13b64e6b9fe75a976e5dd4f0.mp4',
 '/mnt/elice/dataset/test/010fc942573e4679b755acfc1205d91a.mp4',
 '/mnt/elice/dataset/test

In [9]:
import pandas as pd
submission = pd.read_csv('sample_submission.csv')
submission

,path,label
0,000ee59f529d4d069edee33b887bb2ce.mp4,real
1,0018214f560f4ba3871dd9218f55dd48.mp4,real
2,00241f732f934e2b807cf493d02ff80b.mp4,fake
3,00473872f36940a0bfe3f3dfd7667449.mp4,real
4,004d9c5bee68438397889c71a32a05d5.mp4,real
...,...,...
2995,ffa0960e6c154c478b6b1781bda78e08.mp4,real
2996,ffa34ed0124f459885b99401fc2f706c.mp4,real
2997,ffd9e5c5e71a4c33a0b63d3676d15d52.mp4,real
2998,ffde5e9809f34b43ae7decca08f37d0c.mp4,fake


### 정답 label 기록

모델이 추론한 label로 submission의 label을 채워넣습니다.

In [22]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(256),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [19]:
# 데이터셋 및 데이터 로더 설정
test_dataset = VideoDataset(directory=test_files, transform=test_transform, mode='test')
total_size = len(test_dataset)

test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print(total_size )

3000


In [20]:

model_resnet50 = timm.create_model('res2net50_26w_4s', pretrained=True, num_classes=2).to(device)
model_resnet50.load_state_dict(torch.load('resnet_best_model_0.9726.pt', map_location=device))

model_inc4 = timm.models.inception_v4(pretrained=True,num_classes=2).to(device)
model_inc4.load_state_dict(torch.load('incv4_model_0.974.pt', map_location=device))

model = timm.create_model('xception', pretrained=True, num_classes=2).to(device)
model.load_state_dict(torch.load('xception_best_model_0.974.pt', map_location=device))

model2 = timm.models.resnext50d_32x4d(pretrained=True,num_classes=2).to(device)
model2.load_state_dict(torch.load('resnext_best_model_0.975.pt', map_location=device))

model_renest50d = timm.create_model('resnest50d.in1k', pretrained=True, num_classes=2).to(device)
model_renest50d.load_state_dict(torch.load('resnest_best_model_0.976.pt', map_location=device))

# fft
model_twin = VideoClassifier(num_classes=2).to(device)
model_twin.load_state_dict(torch.load('best_model_twin_0.972.pt', map_location=device))





/home/elicer/.local/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


<All keys matched successfully>

In [28]:
import torch.fft as fft# 모델을 평가 모드로 설정
model.eval()
model_inc4.eval()
model_resnet50.eval()
model_renest50d.eval()
model2.eval()
model_twin.eval()

# 예측 결과를 저장할 리스트 초기화
predictions = []

# 테스트 데이터셋 순회
with torch.no_grad():
    for data in tqdm(test_loader, desc="Testing"):
        inputs, fft_input = data[0].to(device),data[1].to(device)
        outputs1 = model(inputs)
        outputs2 = model_inc4(inputs)
        outputs3 = model_resnet50(inputs)
        outputs4 = model_renest50d(inputs)
        outputs5 = model2(inputs)
        outputs6 = model_twin(inputs, fft_input)
        
        outputs = (outputs1 + outputs2 + outputs3 + outputs4  + outputs5  + outputs6)/6

        # 예측된 클래스를 가져옵니다 (예: softmax의 최대값)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

Testing: 100%|██████████| 1500/1500 [05:50<00:00,  4.28it/s]


In [29]:
converted_predictions = ['fake' if pred == 1 else 'real' for pred in predictions]
converted_predictions

['real',
 'real',
 'fake',
 'real',
 'real',
 'real',
 'real',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'real',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'real',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'real',
 'real',
 'fake',
 'real',
 'real',
 'fake',
 'fake',
 'real',
 'real',
 'real',
 'fake',
 'fake',
 'fake',
 'fake',
 'real',
 'real',
 'fake',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'real',
 'real',
 'real',
 'real',
 'fake',
 'fake',
 'fake',
 'fake',
 'real',
 'real',
 'real',
 'real',
 'real',
 'real',
 'fake',
 'fake',
 'real',
 'real',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'real',
 'real',
 'fake',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'real',
 'fake',
 'real',
 'fake',
 'real',
 'fake',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 'fake',
 'fake',
 'real',
 

### 제출파일 생성

정답을 채워넣은 submission 파일을 제출 파일로 생성합니다.

In [30]:
submission['label'] = converted_predictions
submission

,path,label
0,000ee59f529d4d069edee33b887bb2ce.mp4,real
1,0018214f560f4ba3871dd9218f55dd48.mp4,real
2,00241f732f934e2b807cf493d02ff80b.mp4,fake
3,00473872f36940a0bfe3f3dfd7667449.mp4,real
4,004d9c5bee68438397889c71a32a05d5.mp4,real
...,...,...
2995,ffa0960e6c154c478b6b1781bda78e08.mp4,real
2996,ffa34ed0124f459885b99401fc2f706c.mp4,real
2997,ffd9e5c5e71a4c33a0b63d3676d15d52.mp4,real
2998,ffde5e9809f34b43ae7decca08f37d0c.mp4,fake


In [31]:
submission['label'] = converted_predictions
submission['label'].value_counts()

real    1543
fake    1457
Name: label, dtype: int64

In [32]:
submission.to_csv('sample_submission.csv', index=False)

### 제출

우측 상단의 제출 버튼을 눌러, `code.ipynb` 파일과 `sample_submission.csv` 파일을 제출합니다.